<h1 style="background-color:black;font-family:segoe ui;color:#FFF9ED;font-size:300%;text-align:center;border-radius:10px 10px;"> 
🌐 הסבת גושי קדסטר לבנק"ל מודרני 🌐

In [ ]:
print('1. Blocks Borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as daskgpd
from configs import CNFG
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  1.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
טבלת גושים מאורקל

In [ ]:
SubGush_table_cols = ['SUB_GUSH_ID', 'GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'STATUS_DATE', 'HESDER_LAST_PARCEL']
SubGush_table = pd.read_csv(CNFG.Oracle + 'sub_gush.csv', usecols = SubGush_table_cols, encoding = 'cp1255')

<h1 style="text-align:right;font-size:100%">
נתוני שכבת גושים

In [ ]:
SubGush_layer_cols = ['SUB_GUSH_ID_1', 'GUSH_NUM', 'GUSH_SUFFIX', 'geometry']

SubGush_layer = gpd.read_file(CNFG.Arcstorm + 'SUB_GUSH_ALL.gdb', layer = 'SUB_GUSH_ALL', include_fields = SubGush_layer_cols)

SubGush_layer.rename(columns = {'SUB_GUSH_ID_1' : 'SUB_GUSH_ID'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
שכבת מפתח גליונות גושים

In [ ]:
SheetKey_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'geometry']
SheetKey = daskgpd.read_file(CNFG.Arcstorm + 'SHEET_K.gdb', layer = 'SHEET_K', columns = SheetKey_cols, npartitions = 5).compute()
SheetKey.rename(columns = {'geometry' : 'Sheet_geometry'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
טבלת חלקות אחרונות בגושים מאורקל

In [ ]:
SubMMG_cols = ['SUB_GUSH_ID', 'LAST_PARCEL', 'MMG_LAST_PARCEL', 'COURT_LAST_PARCEL']
SubMMG = pd.read_csv(CNFG.Oracle + 'sub_gush_mmg.csv', usecols = SubMMG_cols, encoding = 'cp1255')

<h1 style="text-align:right;font-size:100%">
טבלת גושים שעודכנו בבנק"ל קיים

In [ ]:
edit_dates_cols = ['STATUS_DATE', 'EDIT_ID']
edit_dates = pd.read_csv(CNFG.Oracle + 'PARCEL_EDIT.csv', usecols = edit_dates_cols, parse_dates = ['STATUS_DATE'], encoding = 'cp1255')
edit_dates.rename(columns = {'STATUS_DATE' : 'UpdateDate'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
טבלת תאריכי עדכון בבנק"ל

In [ ]:
edit_blocks_cols = ['SUB_GUSH_ID', 'EDIT_ID']
edit_blocks = pd.read_csv(CNFG.Oracle + 'PARCEL_EDIT_GUSH.csv', usecols = edit_blocks_cols)

In [ ]:
del [SubGush_layer_cols, SubGush_table_cols, SheetKey_cols, SubMMG_cols, edit_dates_cols, edit_blocks_cols]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  1.b Cleaning data')

<h1 style="text-align:right;font-size:100%">
הסרת גושים מכל מקורות המידע שמספר הגוש שלהם הוא 0 או ריק

In [ ]:
SubGush_layer = SubGush_layer[(SubGush_layer['GUSH_NUM'] != 0) | (~SubGush_layer['GUSH_NUM'].isna())]
SubGush_table = SubGush_table[(SubGush_table['GUSH_NUM'] != 0) | (~SubGush_table['GUSH_NUM'].isna())]
SheetKey      = SheetKey     [(SheetKey['GUSH_NUM'] != 0)      | (~SheetKey['GUSH_NUM'].isna())]

<h1 style="text-align:right;font-size:100%">
מילוי ערכים ריקים של שדה תת-גוש בערך 0 לכל מקורות המידע

In [ ]:
SubGush_layer['GUSH_SUFFIX'] = SubGush_layer['GUSH_SUFFIX'].fillna(0)
SubGush_table['GUSH_SUFFIX'] = SubGush_table['GUSH_SUFFIX'].fillna(0)
SheetKey['GUSH_SUFFIX']      = SheetKey['GUSH_SUFFIX'].fillna(0)

<h1 style="text-align:right;font-size:100%">
הסרת סטאטוסים לא ידועים או גושי איו"ש מטבלת גושים באורקל 

In [ ]:
SubGush_table = SubGush_table[~SubGush_table['STATUS'].isin([0, 99, None, np.nan])]

<h1 style="text-align:right;font-size:100%">
הגדרות שדות של תאריכים לסוג תאריך

In [ ]:
SubGush_table['STATUS_DATE'] = pd.to_datetime(SubGush_table['STATUS_DATE'], errors = 'coerce')

<h1 style="text-align:right;font-size:100%">
יצירת שדה מקשר של שם גוש מלא לכל מקורות המידע על מנת להוציא גושים משכבת מפתח גליונות שכבר נמצאים בשכבת גושים

In [ ]:
SubGush_table[['GUSH_NUM', 'GUSH_SUFFIX']] = SubGush_table[['GUSH_NUM', 'GUSH_SUFFIX']].astype(int)
SubGush_layer[['GUSH_NUM', 'GUSH_SUFFIX']] = SubGush_layer[['GUSH_NUM', 'GUSH_SUFFIX']].astype(int)
SheetKey[['GUSH_NUM', 'GUSH_SUFFIX']] = SheetKey[['GUSH_NUM', 'GUSH_SUFFIX']].astype(int)

SheetKey['BlockName'] = SheetKey['GUSH_NUM'].astype(str) + '-' + SheetKey['GUSH_SUFFIX'].astype(str)
SubGush_layer['BlockName'] = SubGush_layer['GUSH_NUM'].astype(str) + '-' + SubGush_layer['GUSH_SUFFIX'].astype(str)

SheetKey = SheetKey[~SheetKey['BlockName'].isin(SubGush_layer['BlockName'])]

SheetKey.drop(columns = 'BlockName', inplace = True)
SubGush_layer.drop(columns = 'BlockName', inplace = True)

In [ ]:
print('  1.c Data conversion')

<h1 style="text-align:right;font-size:100%">
מיזוג כל הנתונים
<h1 style="text-align:right;font-size:75%">
(ישנם מספרי מזהה של גושים בטבלת גושים של אורקל שלא קיים להם מידע במקורות אחרים, גושים אלו יושמטו באיחוד

In [ ]:
Blocks_all = SubGush_table.merge(right = SubGush_layer, on = ['SUB_GUSH_ID' , 'GUSH_NUM', 'GUSH_SUFFIX'], how = 'left')\
                          .merge(right = SheetKey, on = ['GUSH_NUM', 'GUSH_SUFFIX'], how = 'left')\
                          .merge(right = SubMMG, on = 'SUB_GUSH_ID', how = 'left')

del [SubGush_table, SubMMG, SheetKey]

<h1 style="text-align:right;font-size:100%">
מילוי ערכי ריקים של שדה גיאומטריה בעזרת גיאומטריה שמגיעה ממפתח גליונות
<h1 style="text-align:right;font-size:100%">
קיבוץ מרחבי לפי שדה של מספר מזהה לגוש בכדי למנוע יישויות גושים מפוצלים (מולטי-פארטס)

In [ ]:
Blocks_all['geometry'] = np.where(Blocks_all['geometry'].isna(),
                                  Blocks_all['Sheet_geometry'],
                                  Blocks_all['geometry'])

Blocks_all.drop(columns = ['Sheet_geometry'], inplace = True)

Blocks_all = gpd.GeoDataFrame(data = Blocks_all, geometry = 'geometry', crs = 2039)
Blocks_all = Blocks_all.dissolve(by = 'SUB_GUSH_ID', as_index = False, dropna = False)

<h1 style="text-align:right;font-size:100%">
    הגדרת שדה סטאטוס חדש לגוש בעזרת מיפוי ערכים 

In [ ]:
Status_classifier = {
                     1    : 12, #חדש רשום = נוצר בתצ"ר
                     3    : 13, #לא רשום = טרום תצ"ר
                     4    : 27, # לא מוסדר = זמני
                     5    : 21, # מבוטל = נוצר בהסדר 
                     6    : 11, # מוסדר = נוצר בהסדר
                     7    : 22, # בהסדר = בהסדר חלוקה
                     9    : 27, # רישום ראשון = זמני
                     10   : 22, # חלוקה = בהסדר חלוקה
                     11   : 23, # מוקדמת = בהסדר מוקדמת
                     13   : 24, # ארעית = בהסדר ארעית
                     15   : 25, # סופית = בהסדר סופית
                     16   : 11, # ירדני = נוצר בהסדר
                     17   : 21, # ירדני בהסדר = לא מוסדר
                     18   : 21, # רישום בשטח לא מוסדר = לא מוסדר
                     51   : 21, # חדש מבוטל = לא מוסדר
                     56   : 21, # הסדר מבוטל = לא מוסדר
                     57   : 21, # מתבטל הסדר = לא מוסדר
                     58   : 21, # גוש בשטח לא מוסדר - מבוטל = לא מוסדר
                     None : 22 # ללא ערך = זמני
                    }

Blocks_all['BlockStatus'] = Blocks_all['STATUS'].map(Status_classifier)
Blocks_all['BlockStatus'] = Blocks_all['BlockStatus'].astype(int)

del Status_classifier

<h1 style="text-align:right;font-size:100%">
STATUS הגדרת שדה סוג מקרקעין  לגוש בעזרת מיפוי ערכי שדה

In [ ]:
registered_LandTypes   = [11, 12] # or by STATUS [1,3,6,16] = [חדש רשום, לא רשום, מוסדר, ירדני]

register_code   = 1
unregister_code = 2

registered_cond = Blocks_all['BlockStatus'].isin(registered_LandTypes)

Blocks_all['LandType'] = np.where(registered_cond,
                                  register_code,
                                  unregister_code)

del [registered_LandTypes, register_code, unregister_code]

<h1 style="text-align:right;font-size:100%">
STATUS הגדרת שדה בינארי לגוש ירדני בעזרת מיפוי ערך 16 בשדה

In [ ]:
IsJordanian_code    = 1
IsNotJordanian_code = 0

Blocks_all['IsJordanian'] = np.where(Blocks_all['STATUS'] == 16, IsJordanian_code, IsNotJordanian_code)

del [IsJordanian_code, IsNotJordanian_code]

In [ ]:
Blocks_all.drop(columns = 'STATUS', inplace = True)

<h1 style="text-align:right;font-size:100%">
מיזוג טבלאות של עדכנוי גושים בבנק"ל והסרת כפילויות כאשר הרשומה בעלת התאריך המקסימלי תישאר

In [ ]:
edit_dates['UpdateDate'] = pd.to_datetime(edit_dates['UpdateDate'], errors = 'coerce')

update_dates = edit_dates.merge(right = edit_blocks,
                                on    = 'EDIT_ID',
                                how   = 'inner')\
                         .drop(columns = 'EDIT_ID')\
                         .sort_values(by = 'UpdateDate')

update_dates = update_dates.groupby(by = 'SUB_GUSH_ID',
                                    as_index = False,
                                    dropna = True).max()


del [edit_dates, edit_blocks]

<h1 style="text-align:right;font-size:100%">
מיזוג עם נתוני גושים לקבלת תאריכי עדכון אחרון של הגוש

In [ ]:
Blocks_all = Blocks_all.merge(right = update_dates,
                              how   = 'left',
                              on    = 'SUB_GUSH_ID')

del update_dates

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  1.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("BlockUniqueID", int),
                        ('BlockNumber', int),
                        ("SubBlockNumber", int),
                        ("Name", str),
                        ("BlockStatus", int),
                        ("StatedAreaUnit", int),
                        ("IsJordanian", int),
                        ("LandType", int),
                        ("SetteledDate", 'datetime64[D]'),
                        ("UpdateDate", 'datetime64[D]'),
                        ("LastRegisterdParcel", int),
                        ("LastParcel", int),
                        ("LastSetteledParcel", int),
                        ("LastCourtParcel", int),
                        ("RetiredByRecord", str)
                        ])


Blocks = pd.DataFrame(np.empty(0, dtype = new_columns))

<h1 style="text-align:right;font-size:100%">
איכלוס שדות לגושים מוסדרים ולא מוסדרים

In [ ]:
Blocks['BlockUniqueID'] = Blocks_all['SUB_GUSH_ID'].astype(int)


Blocks['BlockNumber'] = Blocks_all['GUSH_NUM'].astype(int)


Blocks['SubBlockNumber'] = Blocks_all['GUSH_SUFFIX'].fillna(0)
Blocks['SubBlockNumber'] = Blocks['SubBlockNumber'].astype(int)


Blocks['Name'] = Blocks['BlockNumber'].astype(str) + '/' + Blocks['SubBlockNumber'].astype(str)


Blocks['BlockStatus'] = Blocks_all['BlockStatus'].astype(int)


Blocks['StatedAreaUnit'] = 109404 #Square Meter


Blocks['IsJordanian'] = Blocks_all['IsJordanian'].astype(int)


Blocks['LandType'] = Blocks_all['LandType'].astype(int)


Blocks['SetteledDate'] = Blocks_all['STATUS_DATE']


Blocks['UpdateDate'] = Blocks_all['UpdateDate']


Blocks['LastRegisterdParcel'] = Blocks_all['LAST_PARCEL'].astype('Int64')


Blocks['LastParcel'] = Blocks_all['MMG_LAST_PARCEL'].astype('Int64')


Blocks['LastSetteledParcel'] = Blocks_all['HESDER_LAST_PARCEL'].astype('Int64')


Blocks['LastCourtParcel'] = Blocks_all['COURT_LAST_PARCEL'].astype('Int64')


Blocks['geometry'] = Blocks_all['geometry']


Blocks['RetiredByRecord'] = np.where(Blocks['geometry'].isna(), CNFG.FabricatedCancelProcessGUID, None)

Blocks['CreatedByRecord'] = CNFG.FabricatedCreateProcessGUID

<h1 style="text-align:right;font-size:100%">    
מיון הטבלה לפי מספר מזהה גוש

In [ ]:
Blocks.sort_values('BlockUniqueID', inplace = True)

del Blocks_all

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות לגושים מוסדרים/לא מוסדרים 💾
<h1 style="text-align:right; font-size:100%;">
<h1 style="text-align:right; font-size:100%;">     
שמירת טבלה מצומצמת של מספרי זיהוי לצורך שימוש בהסבתשכבות בהמשך לקבלת מספרי מזהה לגוש
<h1 style="text-align:right; font-size:100%;">     
פיצול התוצאות לפי יישויות בעלי גיאומטריה וללא גיאומטריה    
<h1 style="text-align:right; font-size:100%;"> 
שמירת טבלת של כל הגושים ללא גיאומטריה
<h1 style="text-align:right; font-size:100%;"> 
שמירת שכבת גיאוגרפית של כל הגושים בעלי גיאומטריה
<h1 style="text-align:right; font-size:100%;"> 
הפעלת יישום להכנסת כל הגושים לטבלת גושים בבנק"ל

In [ ]:
print('  1.e Exporting results to product folder')

In [ ]:
Blocks[['BlockNumber', 'SubBlockNumber', 'Name', 'BlockUniqueID', 'LandType']].to_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', index = False)

In [ ]:
Geometry      = gpd.GeoDataFrame(Blocks[~Blocks['geometry'].isna()], geometry = 'geometry', crs = 2039)

EmptyGeometry = Blocks[Blocks['geometry'].isna()].drop(columns = 'geometry')

In [ ]:
json_dir        = CNFG.JsonPath + 'Blocks_geom.geojson'
no_geom_csv_dir = CNFG.CSVPath  + 'Blocks_no_geom.csv'

Geometry.to_file(json_dir, driver = 'GeoJSON')
EmptyGeometry.to_csv(no_geom_csv_dir , index = False)

In [ ]:
subprocess.call([CNFG.BatchPath + 'Blocks-Save as Feature Class.bat']);